In [ ]:
'''
this script combine the selected movies both in movielens and having scripts,
i) the selected genome of movies are found in '../movielens/ml-latest/select-genome-scores.csv'
ii) selected movie is found in '../movielens/ml-latest/select-movies.csv'
------
the progression to combine:
1) create movie title2id dict and saved as title2id.pkl.
2) crate a dict of movie title to id and saved as '1title2id.pkl'
3)read the emotional arc similarities and genome similarities from c++ calculation results and saved them as npy files.
------
'''

In [14]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle
from sklearn.metrics.pairwise import cosine_similarity
genome=pd.read_csv('../movielens/ml-latest/select-genome-scores.csv')
movie=pd.read_csv('../movielens/ml-latest/select-movies.csv')


In [73]:
movie.shape

(9255, 4)

In [30]:
def create_title2id_table(movie):
    #movie=movie.reset_index()
    title2id={}
    movie_title_without_year=movie['title_without_year'].unique().tolist()
    for i in movie_title_without_year:
        sel=movie[movie['title_without_year']==i]
        if sel.shape[0]==1:
            title2id[i]={'title':sel['title'].values[0],'id':sel['movieId'].values[0]}
        else:
            try:
                sel['year']=sel['title'].apply(lambda x :int(x[-5:-1]))
                latest=sel['year'].max()
                latest_movie=sel[sel['year']==latest]
                title2id[i]={'title':latest_movie['title'].values[0],'id':latest_movie['movieId'].values[0]}

            except ValueError:
                latest_movie=sel.iloc[0,:]
                title2id[i]={'title':latest_movie['title'],'id':latest_movie['movieId']}
    return title2id


In [34]:
create_title2id_table=create_title2id_table(movie)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
with open('/Users/yangyangwang/PycharmProjects/movie_project/app/static/mood/movies.pkl','wb') as f:
        pickle.dump(movie, f, pickle.HIGHEST_PROTOCOL)

In [35]:
with open('/Users/yangyangwang/PycharmProjects/movie_project/app/static/mood/title2id.pkl','wb') as f:
        pickle.dump(create_title2id_table, f, pickle.HIGHEST_PROTOCOL)

In [62]:
##read from the result of c++ calculations

genome_similarities=pd.read_csv('../data/for_c++_calculation/genome_similarities.csv',header=None).drop(3957,axis=1)
genome_similarities.head()

,0,1,2,3,4,5,6,7,8,9,...,3947,3948,3949,3950,3951,3952,3953,3954,3955,3956
0,1.000000,0.773108,0.701040,0.625887,0.534362,0.768296,0.583764,0.583224,0.616548,0.601451,...,0.558246,0.526866,0.724897,0.595291,0.721148,0.759237,0.752689,0.743733,0.571849,0.702102
1,0.773108,1.000000,0.680026,0.520927,0.634564,0.667072,0.521917,0.710084,0.506195,0.527953,...,0.507594,0.495829,0.659659,0.706509,0.662483,0.651703,0.642089,0.685301,0.522494,0.604994
2,0.701040,0.680026,1.000000,0.636414,0.613296,0.598888,0.601501,0.654646,0.603596,0.614910,...,0.600070,0.567168,0.688645,0.660141,0.702621,0.709137,0.693452,0.746736,0.607965,0.641435
3,0.625887,0.520927,0.636414,1.000000,0.597719,0.479936,0.670502,0.540400,0.874655,0.575067,...,0.793295,0.606873,0.642751,0.544495,0.594271,0.864854,0.658625,0.541178,0.605767,0.786829
4,0.534362,0.634564,0.613296,0.597719,1.000000,0.472036,0.499424,0.735319,0.502073,0.429216,...,0.584473,0.456803,0.550349,0.664022,0.556555,0.654543,0.510762,0.556752,0.512169,0.562402


In [55]:
sentiment_similarities=pd.read_csv('../data/for_c++_calculation/sentiment_line_similarities2.csv',header=None).drop(3767,axis=1)
sentiment_similarities.head()

,0,1,2,3,4,5,6,7,8,9,...,3757,3758,3759,3760,3761,3762,3763,3764,3765,3766
0,1.000000,-0.303180,0.181183,-0.574727,0.175850,0.075475,-0.011222,-0.875789,0.476213,0.459588,...,0.459577,0.395016,-0.244220,0.962523,-0.021995,-0.543586,0.305779,-0.476839,-0.687140,-0.349482
1,-0.303180,1.000000,0.190407,0.691417,0.467854,-0.143197,-0.432927,0.348177,-0.881112,0.433388,...,0.236838,-0.160026,0.179499,-0.175318,0.843631,0.247257,0.051843,-0.365887,0.215994,0.432388
2,0.181183,0.190407,1.000000,-0.304949,0.736840,-0.369059,0.531072,0.010066,0.180922,0.496973,...,-0.097423,-0.525876,-0.593410,0.108467,0.625665,-0.436166,0.979426,-0.848923,0.378195,-0.396594
3,-0.574727,0.691417,-0.304949,1.000000,0.257420,0.190481,-0.614865,0.278499,-0.807353,0.050670,...,0.309840,-0.258449,0.612344,-0.341588,0.284868,0.228750,-0.385170,0.252951,0.014025,0.783903
4,0.175850,0.467854,0.736840,0.257420,1.000000,-0.170226,0.064130,-0.245903,-0.186697,0.724795,...,0.349067,-0.528530,-0.064789,0.294218,0.690282,-0.520013,0.730490,-0.752425,0.008849,0.069753


In [52]:
sentiment_similarities.shape

(3767, 3768)

In [63]:
np.save('../data/genome_similarities_Matrix.npy',genome_similarities)


In [56]:
np.save('../data/sentiment_similarities_Matrix.npy',sentiment_similarities)


In [57]:
#t=np.load('/Users/yangyangwang/PycharmProjects/movie_project/app/static/mood/sentiment_similarities_Matrix.npy')

In [65]:
#t=np.load('/Users/yangyangwang/PycharmProjects/movie_project/app/static/mood/sentiment_similarities_Matrix.npy')
